## Cross-Platform verification

In this tutorial, we will show to measure the fidelity $\mathcal{F}=\mathrm{Tr}(\rho_1\rho_2)/\sqrt{\mathrm{Tr}(\rho_1^2)\mathrm{Tr}(\rho_2^2)}$ between two states $\rho_1$, $\rho_2$ prepared on two quantum devices. 

Related papers [Elben et al, PRL 2019](https://doi.org/10.1103/PhysRevLett.124.010504), [Zhu et al, Nat. Comm 2022](https://www.nature.com/articles/s41467-022-34279-5)

In [4]:
using Revise
using ProgressMeter
using ITensors
using RandomMeas

We first consider a state $\rho_1$ prepared by a random quantum circuit

In [5]:
N = 6
ξ = siteinds("Qubit", N)
depth = 2
circuit = random_circuit(ξ, depth)
states = ["Dn" for n in 1:N]
ψ0 = MPS(ξ,states);
ρ0 = outer(ψ0',ψ0);
ρ1 = apply(circuit,ρ0,apply_dag=true);

We then apply a sequence of randomized measurements and collect the measurement data. 

In [7]:
#Perform Randomized measurements
nu=200 #Number of random unitaries
NM=1000 #Number of projective measurements per unitary
u = Vector{Vector{ITensor}}()
data1 = zeros(Int,(nu,NM,N)) #Data storage
@showprogress dt=1 for r in 1:nu
            push!(u,get_rotations(ξ,"Haar")) #generate random unitary
            data1[r,:,:] = get_RandomMeas(ρ1,u[r],NM,"dense") #data acquisation in simulated quantum device
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:06


In our simulated experiment, the second state $\rho_2$ differs from $\rho_1$ due to local depolarization. We sample randomized measurements according to the same random unitaries $u$.

In [8]:
p = 0.1*rand(N)
ρ2 = apply_depo_channel(ρ1,p)
println("state prepared in the `experiment' 2 ")
data2 = zeros(Int,(nu,NM,N))
@showprogress dt=1  for r in 1:nu
                data2[r,:,:] = get_RandomMeas(ρ2,u[r],NM,"dense")
end

state prepared in the `experiment' 2 


In order to extract the statistical correlations between the measurements data, we first evaluate the Born probabilities for each random unitary.

In [9]:
P1 = Vector{ITensor}()
P2 = Vector{ITensor}()
for r in 1:nu
                push!(P1,get_Born(data1[r,:,:],ξ))
                push!(P2,get_Born(data2[r,:,:],ξ))
end

We then use the formula $\mathrm{tr}(\rho_1\rho_2)=(-2)^{-D[s,s']}\sum_{s,s'}P_u^{(1)}(s)P_u^{(2)}(s')$ to extract the overlap (and proceed similarly to access the purities $\mathrm{tr}(\rho_1^2)$, $\mathrm{tr}(\rho_2^2)$)

In [10]:
overlaps = 0
for r in 1:nu
    global overlaps += get_overlap(P1[r],P2[r],ξ,N)/nu
end
purity1s = get_purity_hamming(data1,ξ)
purity2s = get_purity_hamming(data2,ξ)
Fs = overlaps/max(purity1s,purity2s)
println("Estimated overlap ",overlaps)
println("Estimated purity1 ",purity1s)
println("Estimated purity2 ",purity2s)
println("Estimated fidelity ",Fs)

Estimated overlap 0.8521372600000001
Estimated purity1 1.0300545045045042
Estimated purity2 0.7113985285285284
Estimated fidelity 0.8272739513040729


We can now compare these measured values from the expected ones.

In [11]:
overlap = real(inner(ρ1,ρ2))
purity1 = get_purity(ρ1)
purity2 = get_purity(ρ2)
F = overlap/max(purity1,purity2)
println("Overlap ",overlap)
println("Purity1 ",purity1)
println("Purity2 ",purity2)
println("Fidelity ",F)

Overlap 0.8248451732191823
Purity1 1.0000000000000009
Purity2 0.6841706916624143
Fidelity 0.8248451732191815
